<a href="https://colab.research.google.com/github/ArianFiroozi/AI-Course-Projects/blob/master/LLM_CA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CA 1, LLMs Spring 2025

- **Name:** Arian Firoozi
- **Student ID:** 810100196

---
#### Your submission should be named using the following format: `CA1_LASTNAME_STUDENTID.ipynb`.

---

##### *How to do this problem set:*

- Some questions require writing Python code and computing results, and the rest of them have written answers. For coding problems, you will have to fill out all code blocks that say `YOUR CODE HERE`.

- For text-based answers, you should replace the text that says ```Your Answer Here``` with your actual answer.

- There is no penalty for using AI assistance on this homework as long as you fully disclose it in the final cell of this notebook (this includes storing any prompts that you feed to large language models). That said, anyone caught using AI assistance without proper disclosure will receive a zero on the assignment (we have several automatic tools to detect such cases). We're literally allowing you to use it with no limitations, so there is no reason to lie!

---

##### *Academic honesty*

- We will audit the Colab notebooks from a set number of students, chosen at random. The audits will check that the code you wrote actually generates the answers in your notebook. If you turn in correct answers on your notebook without code that actually generates those answers, we will consider this a serious case of cheating.

- We will also run automatic checks of Colab notebooks for plagiarism. Copying code from others is also considered a serious case of cheating.

---

If you have any further questions or concerns, contact the TAs via email: vahyd@live.com / amirh.bonakdar@ut.ac.ir

In [ ]:
!pip install transformers peft datasets accelerate scipy bitsandbytes wandb  -qqq

In [1]:
print("hello world") #testing github api

hello world


### Q0: Setting Up

Create a Huggingface Access Token From:
https://huggingface.co/settings/tokens

You need to request for access to:
- ```meta-llama/Llama-3.2-1B```
- ```meta-llama/Llama-3.2-1B-Instruct```
- ```mistralai/Mistral-7B-v0.1```

In [ ]:
!huggingface-cli login --token {YOUR_HF_TOKEN}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `llm_ut_ca1` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llm_ut_ca1`


In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, TaskType, get_peft_model, PeftModel, PrefixTuningConfig, PromptTuningConfig
import os
import numpy as np

In [ ]:
BASE_MODEL = 'meta-llama/Llama-3.2-1B'
INSTRUCT_MODEL = 'meta-llama/Llama-3.2-1B-Instruct'

In [ ]:
DEVICE = "cpu"
if torch.backends.mps.is_available():
    DEVICE = "mps"
elif torch.cuda.is_available():
    DEVICE = "cuda"

print(f"Using device: {DEVICE}")

Using device: cuda


## Getting Started with LLMs

## Q1: First Steps (25 pts)

The outputs of tokenizer are not human readable.

In [ ]:
model_id = INSTRUCT_MODEL

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=DEVICE,
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
prompt = "What is 2 plus 2?"

inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
)

outputs

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


#### Q1.1: Readable Model Generation (1 pts)

a. As you see the model outputs token ids which are not readable to us. We should decode this to human readable language. Using the ```decode``` function on the tokenizer, print the human readable model generation.

In [ ]:
### Your Code Here

b. The input prompt is still a part of the output, but we only want to see the model generation. Fix this problem.

In [ ]:
### Your Code Here

#### Q1.2: Generation Function (1 pts)

a. Write and test a function that takes the model, generation config as kwargs with default values, tokenizer and prompt as input and outputs the model generation (generation only). You will be using this in the next sections quite a lot.

In [ ]:
## Your Code Here

#### Q1.3: Comparing different Tokenizers (3 pts)

a. Bring in the tokenizer for:

- ```meta-llama/Llama-3.2-1B```
- ```mistralai/Mistral-7B-v0.1```
- ```microsoft/Phi-4-mini-instruct```.

Tokenize a PERSIAN sentence with at least 10 words using the tokenizers of all three models from different families and print the human readable output.

In [ ]:
### Your Code Here

b. Compare the outputs, Which one produces better tokens? What is the reason for this difference in tokenization?

```Your Answer Here```

#### Q1.4: Base Model vs. Instruction-tuned Model (10 pts)

a. See the difference between Base and Instruct Models using the prompt ```What is 2+2?```, Keep in mind that when temperature != 0, you will get different answers. Generate the answers a few time to get a sense of how models work.

***NOTE:*** It is recommended to play with various prompts and generation configs.

In [ ]:
## Your Code Here

b. In a concise way, what is the difference in outputs? Why the models answer the way they do and how does it affect the way we prompt them?

```Your Answer Here```

#### Q1.5: Chat Templates for Instruct Models (10 pts)

When using multi-turn or complex chats with LLMs, to maintain context and keep the generation controlled, it is a good practice to comply with the instruction format used by models. Previous instruction-tuned models needed this to do even the simplest tasks but the recent ones are mostly robust to it and can work without it in simple examples. In this section we will go over this concept.


An Instruction (Chat) template generally has 3+1 main components (roles):
- System Instruction aka system role
- User Query aka user role
- LLM Answer aka assistant role
- (Tool Calls)

```apply_chat_template``` on huggingface tokenizers is a unified interface for chat templates used by different models. The providers are responsible for defining this on the tokenizer according to the template they have used during training stage.

a. Bring in the tokenizer and print the ```chat_template``` property on it.

In [ ]:
### Your Answer Here

b. In maximum two sentences, what do you see and what is this? How it is used?

```Your Answer Here```

c. Organize the content below using system and user prompt in standard ```ChatML``` format (list of dicts with certain keys), transform them to the instruction format used by LLaMa 3 Models using the ```apply_chat_template``` function and print the human readable output.

**System:** You are a funny math teacher, you should answer math questions in a playful and funny tone.

**User:** What is 2+2

***NOTE:*** You can use ```skip_special_tokens = True``` when decoding to get rid of template tags. You also may update the generate function from previous steps and use that.

In [ ]:
### Your Code Here

d. Now prompt the model with and without chat template being applied. (In second scenario simply put the system prompt followed by a newline and the user querry as one single string)

In [ ]:
### Your Code Here

e. What is your observation, do we need instruction formats in this scenario or the model can follow?

```Your Answer Here```

Well, let's go a step further :)

Below is a conversation between Dr. Yaghoobzadeh and Dr. Dousti (The content is generated by LLMs and I don't know what's going on in this conversation)

In [ ]:
# Conversation data ready for applying ChatML template
messages = [
    {
        "role": "system",
        "content": "You are a system that can simulate two AI experts debating: Dr. Dousti (NLP researcher) and Dr. Yaghoobzadeh (NLP researcher). When I write [DOUSTI], respond as Dr. Dousti. When I write [YAGHOOBZADEH], respond as Dr. Yaghoobzadeh. Both should maintain their distinct perspectives and expertise."
    },
    {
        "role": "user",
        "content": "[DOUSTI] What's your assessment of retrieval-augmented generation for LLMs in production systems?"
    },
    {
        "role": "assistant",
        "content": "As Dr. Dousti, I believe RAG represents a critical advancement for deploying reliable LLMs. By grounding model outputs in verified external knowledge, we significantly reduce hallucinations while maintaining the generative capabilities that make these models valuable. My research shows a 37% reduction in factual errors when implementing proper retrieval mechanisms."
    },
    {
        "role": "user",
        "content": "[YAGHOOBZADEH] But what about the computational overhead and latency issues for real-time applications?"
    },
    {
        "role": "assistant",
        "content": "As Dr. Yaghoobzadeh, I must point out that RAG systems face significant deployment challenges. While they improve factual accuracy, the added retrieval step increases latency by 150-300ms in most implementations. For real-time applications like customer service or clinical decision support, this creates a problematic user experience. Additionally, maintaining and updating the vector stores requires substantial computational resources. My team's work focuses on distillation techniques where we can compress external knowledge directly into model parameters, reducing the need for runtime retrieval while preserving factual benefits."
    },
    {
        "role": "user",
        "content": "[DOUSTI] Do you think modular architectures with specialized retrievers are the answer?"
    }
]

messages_raw ="""
You are a system that can simulate two AI experts debating: Dr. Dousti (NLP Researcher) and Dr. Yaghoobzadeh (NLP Researcher). When I write [DOUSTI], respond as Dr. Dousti. When I write [YAGHOOBZADEH], respond as Dr. Yaghoobzadeh. Both should maintain their distinct perspectives and expertise.
[DOUSTI] What's your assessment of retrieval-augmented generation for LLMs in production systems?
As Dr. Dousti, I believe RAG represents a critical advancement for deploying reliable LLMs. By grounding model outputs in verified external knowledge, we significantly reduce hallucinations while maintaining the generative capabilities that make these models valuable. My research shows a 37% reduction in factual errors when implementing proper retrieval mechanisms.
[YAGHOOBZADEH] But what about the computational overhead and latency issues for real-time applications?
As Dr. Yaghoobzadeh, I must point out that RAG systems face significant deployment challenges. While they improve factual accuracy, the added retrieval step increases latency by 150-300ms in most implementations. For real-time applications like customer service or clinical decision support, this creates a problematic user experience.
[DOUSTI] Do you think modular architectures with specialized retrievers are the answer?
"""


f. Now repeat what you have done with funny teacher example and compare the results with and without applying chat template.

In [ ]:
### Your Code Here

g. Write your observations down here. Does the model comply to what we want without using templates in this scenario? Why?

```Your Answer Here```

## Q2: Fine-tuning using LoRa (75 pts)

Let's make it more interesting. We certainly don't want to just prompt models here. We will fine-tune a base model using a small classification dataset on emotion detection. The resulting model's performance will be compared with the instruction-tuned model by Meta and the base model. We will get a sense of how everything works quantitively. We don't want you to just stare at the screen watching the model converge. With the right configurations, your training should not take more than 10 minutes and the purpose here is for you to learn a diverse set of tools that will help you in doing your final project.

### A. Dataset (15 pts)

In [ ]:
DS_NAME = 'emotion'
DS_TRAINING_SIZE = 1500
DS_TEST_SIZE = 100
DS_VALIDATION_SIZE = 50

a. Read the dataset from huggingface. Look at the features and the distribution on the labels of the dataset to get a sense of what it is about.

In [ ]:
### Your Code Here

#### Q2.0: Utilities (5 pts)

a. Write a function named `get_stratified_sample` that takes the following parameters:
- `dataset`: The input dataset (a Hugging Face Dataset object).
- `n_samples`: The desired number of samples in the stratified sample.
- `random_state`: An integer for reproducible sampling (default to 42).

The function should return a stratified sample of the dataset, maintaining the original class proportions.

Keep in mind that we need ```DS_TRAINING_SIZE``` samples for training and ```DS_TEST_SIZE``` samples for testing. If you are going to use the validation set, ```DS_VALIDATION_SIZE``` is needed for this. You may change these if you see fit but with these numbers, you can get a good enough result in an acceptable time.

***NOTE:*** Make sure your function shuffles the final dataset.

In [ ]:
### Your Code Here

b. Use your function to create train,test and (validation) sets. Compare the distribution of labels with the full dataset to make sure it's working correctly. Printing or plotting the distributions is enough.

In [ ]:
### Your Code Here

#### Q2.1: Preparing Data for Fine-Tuning (10 pts)

a. Let's get the emotion dataset ready for LoRA fine-tuning. Here's what you need to do:

1.  **Format the Data**: Turn each data entry into a conversation like this:
    *   A system instruction that tells the model what to do (analyze emotions)```*```.
    *   A user query that gives the model the text to analyze.
    *   An assistant response that provides the correct emotion label (in natural language, naturally!)
2.  **Tokenize and Label**:
    *   Tokenize the formatted conversation.
    *   Prepare labels for training, make sure to mask the instruction part of the data ```**```.

Also, write a verification function that in a human readable format:

*   Prints the complete training input sequence after tokenization for a given data entry.
*   Shows the labels, indicating which tokens are being predicted.
*   Checks if the assistant header is correctly handled by finding its position in the text and printing the subsequent text.

```*TIP:``` It is a good practice to make your system instruction as concise as possible. For example in this task, you should tell the LLM explicitly that what are the valid labels.

b. When preparing the data, experiment with the tokenizer parameters, namely `truncation`, `padding` and `max_length`. In a ```concise``` manner, explain what each one of them does and what is a good value and why.

```Your Answer Here```

c. ```**```When preparing the data, mask the instruction part of the data (set labels to -100 for the instruction tokens) before starting the training. Why is this a good idea?

```Your Answer Here```

In [ ]:
### Your Code Here

d. Run your verification function on the first sample of your training dataset to see everything is in order.

In [ ]:
### Your Code Here

### B. Fine-tune using LoRa (30 pts)

In [ ]:

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


#### Q2.2: Experimenting with LoRA Configuration Parameters (3 pts)

In this section, you may explore the effect of different LoRA configuration parameters on the trainable parameter count:

1. Try different rank values (`r`) - experiment with values like 8, 16, 32, and 64
    - Higher rank allows for more expressive power but increases parameter count
    
2. Adjust the scaling factor (`lora_alpha`) - typically set to 2x the rank
    - This affects the magnitude of updates during training
    
3. Modify target modules - test different combinations like:
    - Only attention modules: `["q_proj", "v_proj"]`
    - All attention modules: `["q_proj", "k_proj", "v_proj", "o_proj"]`
    - Including feed-forward: `["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]`
    
4. Vary dropout rates (`lora_dropout`) - test values like 0.0, 0.05, 0.1
    - Higher dropout can help with regularization

You may use the `print_trainable_parameters()` function to observe how each change affects the number of trainable parameters.

(We are not requiring you to print and explain everything, these are some values to help you out)

a. Find a configuration that provides a good balance between parameter efficiency and model expressiveness. Explain your reasons in a concise manner.


```Your Answer Here```

In [ ]:
### Your Answer Here (Final Chosen Lora Config + Output of trainable parameters function on that)

#### Q2.3: Training Callbacks and Early Stopping (10 pts)


**Understanding Training Callbacks**

Generally speaking, in deep learning, callbacks are functions that can be applied at various stages of training
(start/end of training, epoch, or batch) to modify the training process. They're powerful
tools that allow you to:

- Monitor training metrics in real-time
- Add custom logging
- Save model checkpoints
- Implement early stopping
- Adjust learning rates dynamically

**Early Stopping**

Early stopping is a regularization technique that prevents overfitting by stopping training
when a monitored metric stops improving. Benefits include:

- Reduced training time
- Better generalization
- Prevention of overfitting

**Your Task**

a. Implement a custom callback class that:
1. Tracks the best loss value during training
2. Calculates perplexity in steps
3. Adds perplexity to the training logs
4. Implements early stopping if the loss doesn't improve for several steps (This is called patience)
5. (In your final project it is a good idea to use the big enough validation set to better monitor the training process. Given the time constraints for this assignment, we are not requiring you to do that.)

***NOTE:*** You should inherit from the TrainerCallback class implemented in transformers


In [ ]:
### Your Code Here

#### Q2.4: TrainingArgs (7 pts)

b. Explain the purpose of a minimum of 5 of the following TrainingArguments parameters in ```at most two sentences.```
For each parameter, suggest a good value for our emotion classification problem,
considering we are using a Llama-3.2-1B model and training in a Colab/Kaggle environment.
Explain why you chose that value.

1.  `lr_scheduler_type`
2.  `per_device_train_batch_size`
3.  `gradient_accumulation_steps`
4.  `learning_rate`
5.  `weight_decay`
6.  `bf16`
7.  `max_grad_norm`
8.  `warmup_ratio`
9.  `group_by_length`

```Your Answer Here```

b. Define your trainings args

In [ ]:
### Your Code Here

#### Q2.5: Memory usage (8 points)

Now, we want to determine the memory required to **load and train** the LLM in different fine-tuning scenarios.  

- **Full Fine-Tuning:** Calculate the total memory needed when updating all model parameters.  
- **LoRA Fine-Tuning:** Calculate the memory needed based on your LoRA configuration.  
- Use your current settings for the calculations.  
- Refer to [this resource](https://blog.eleuther.ai/transformer-math/) for guidance.

```Your Answer Here```

#### Q2.6: Training the model (2 pts)

Train and save the model. Your training should take at most 10 minutes on a Google colab notebook.

***PRO-TIP:*** If you want to go a step further on a good training task, you may research and use model checkpointing and monitoring tools (like weights and biases and tensorboard) But it's not required here.

In [ ]:
### Your Code Here

### C. Some other PEFT methods (6 pts)

#### Q2.7: IA3 method (2 points)

IA3 ([Liu et al., 2022](https://openreview.net/pdf?id=rBCvMG-JsPd)) is another PEFT method. Briefly explain how it works.

```Your Answer Here```

#### Q2.8: Soft Prompt methods (4 points)

Instead of fine-tuning all model parameters, prompting uses additional input text to guide a frozen model toward a specific task.  

There are two types of prompts [(Hugging Face, PEFT)](https://huggingface.co/docs/peft/en/conceptual_guides/prompting):  
- **Hard prompts**: Manually crafted text prompts using discrete tokens, but designing them is labor-intensive.  
- **Soft prompts**: Learnable tensors concatenated with input embeddings and optimized for a dataset, but they are not human-readable.  

In this section, you will explore how soft prompts are implemented and fine-tuned using PEFT.


Briefly explain the following soft prompt methods and highlight their key differences:  
- **Prompt Tuning** [(Lester et al., 2021)](https://aclanthology.org/2021.emnlp-main.243.pdf)  
- **Prefix Tuning** [(Li & Liang, 2021)](https://aclanthology.org/2021.acl-long.353.pdf)  
- **P-Tuning** [(Liu et al., 2021)](https://arxiv.org/pdf/2103.10385)  

```Your Answer Here```

### D. Evaluate and Comparison (24 pts)

#### Q2.9: Generating Output from Models (10 pts)

Generate the output of models on the task of emotion detection using:

- LoRa fine-tuned Model by you
- Instruction tuned model by Meta
- Base model by Meta

You may use ```Regex``` or simply looking for label names in model outputs to do obtain the classification repots. Looking at the results generated by models can help you greatly to find the best way to parse the output.

***NOTE:*** Your fine-tuned model MUST outperform the base model, but outperforming the instruction tuned model is optional and has extra points. (5 pts)

In [ ]:
### Your Code Here

#### Q2.10: Performance Comparison Visualization (4 pts)

Compare the Accuracy and Micro-F1 in a grouped bar chart. (4 pts)

In [ ]:
### Your Code Here

#### Q2.11: Analysis (10 pts)

Analyze the results and the reasons behind them.

```Your Answer Here```